In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/manhnt27/deep-person-reid-backup.git


Cloning into 'deep-person-reid-backup'...
remote: Enumerating objects: 36600, done.
remote: Counting objects: 100% (36600/36600), done.
remote: Compressing objects: 100% (36567/36567), done.
remote: Total 36600 (delta 30), reused 36600 (delta 30), pack-reused 0
Receiving objects: 100% (36600/36600), 136.25 MiB | 26.34 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Checking out files: 100% (36562/36562), done.


In [6]:
%cd deep-person-reid-backup/

/content/deep-person-reid-backup


In [7]:
! pip install -r requirements.txt


     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 185 kB 48.0 MB/s 
     |████████████████████████████████| 42 kB 948 kB/s 
     |████████████████████████████████| 42 kB 898 kB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████████████████████| 41 kB 604 kB/s 


In [10]:
!pip install torch torchvision torchaudio

In [11]:
!python setup.py develop

/content/deep-person-reid-backup/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Traceback (most recent call last):
  File "setup.py", line 51, in <module>
    long_description=readme(),
  File "setup.py", line 9, in readme
    with open('README.rst') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'README.rst'


In [12]:
import torchreid

/content/deep-person-reid-backup/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


In [13]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='dukemtmcreid',
    targets='dukemtmcreid',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


  **************** Summary ****************
  source            : ['dukemtmcreid']
  # source datasets : 1
  # source ids      : 702
  # source images   : 16522
  # source cameras  : 8
  target            : ['dukemtmcreid']
  *****************************************




In [ ]:
!ls reid-data/dukemtmc-reid/

In [14]:
model = torchreid.models.build_model(
    name='osnet_x0_75',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1uwA9fElHOk3ZogwbeY5GkLI6QPTX70Hq
To: /root/.cache/torch/checkpoints/osnet_x0_75_imagenet.pth
100%|██████████| 7.35M/7.35M [00:00<00:00, 120MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_75_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [15]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [17]:
engine.run(
    save_dir='log/osnet_x0_75',
    max_epoch=50,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/50][10/516]	time 0.371 (0.954)	data 0.001 (0.060)	eta 6:50:03	loss 6.4783 (6.5532)	acc 6.2500 (1.2500)	lr 0.000300
epoch: [1/50][20/516]	time 0.373 (0.664)	data 0.002 (0.031)	eta 4:45:05	loss 6.5859 (6.5195)	acc 0.0000 (2.3438)	lr 0.000300
epoch: [1/50][30/516]	time 0.370 (0.567)	data 0.001 (0.021)	eta 4:03:42	loss 6.4416 (6.4874)	acc 6.2500 (2.5000)	lr 0.000300
epoch: [1/50][40/516]	time 0.371 (0.519)	data 0.000 (0.016)	eta 3:42:46	loss 6.5037 (6.4658)	acc 3.1250 (2.5000)	lr 0.000300
epoch: [1/50][50/516]	time 0.377 (0.490)	data 0.001 (0.013)	eta 3:30:11	loss 6.3636 (6.4315)	acc 3.1250 (2.8125)	lr 0.000300
epoch: [1/50][60/516]	time 0.376 (0.470)	data 0.000 (0.011)	eta 3:21:47	loss 6.2837 (6.4086)	acc 9.3750 (2.9167)	lr 0.000300
epoch: [1/50][70/516]	time 0.374 (0.456)	data 0.001 (0.009)	eta 3:15:45	loss 6.0628 (6.3594)	acc 3.1250 (3.4375)	lr 0.000300
epoch: [1/50][80/516]	time 0.384 (0.446)	data 0.001 (0.008)	eta 3:11:23	loss 6.4672 (6.3449)	acc 3.1250 (3.5547)	lr 0.000300


In [19]:
%cp -av "/content/deep-person-reid-backup" "/content/drive/MyDrive/"

Streaming output truncated to the last 5000 lines.
'/content/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0072963.jpg' -> '/content/drive/MyDrive/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0072963.jpg'
'/content/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0073083.jpg' -> '/content/drive/MyDrive/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0073083.jpg'
'/content/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0073203.jpg' -> '/content/drive/MyDrive/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0073203.jpg'
'/content/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train/4275_c6_f0073323.jpg' -> '/content/drive/MyDrive/deep-person-reid-backup/reid-data/dukemtmc-reid/DukeMTMC-reID/bounding_box_train